In [2]:
!pip install albumentations
!pip install efficientnet_pytorch

     |████████████████████████████████| 634kB 11.1MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp37-none-any.whl size=654019 sha256=386ca7f897a765b099813623e6ed23793b2c87ff5ee338fe95563abfdf435299
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=c45f2c329c5cce0aff138b5a8d3cab92e79f9c0e86feec3d450843dcce12cfab
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch


In [3]:
#импортируем все что пригодится
import numpy as np 
import pandas as pd 
import shutil
import os
import zipfile
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
import torch.nn.functional as F
import copy
import tqdm
import time
import random
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, metrics

import albumentations
from albumentations import pytorch as AT

from efficientnet_pytorch import EfficientNet

%matplotlib inline

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(42)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
with zipfile.ZipFile('/content/drive/MyDrive/taskonML/train.zip', 'r') as zip_ref:
  zip_ref.extractall('/content')

with zipfile.ZipFile('/content/drive/MyDrive/taskonML/test.zip', 'r') as zip_ref:
  zip_ref.extractall('/content')

In [7]:
train_dir = '/content'
test_dir = '/content/test'

In [8]:
train_data = pd.read_csv('/content/drive/MyDrive/taskonML/train_labels.csv')
kl = np.asarray(train_data['sports'].unique())

In [9]:
encoder = LabelEncoder()
train_data['num_labels'] = encoder.fit_transform(train_data['sports'])

In [10]:
train_data

,image,sports,num_labels
0,./train/0.jpg,baseball,1
1,./train/1.jpg,formula1,8
2,./train/2.jpg,fencing,6
3,./train/3.jpg,motogp,13
4,./train/4.jpg,ice_hockey,11
...,...,...,...
11035,./train/11035.jpg,motogp,13
11036,./train/11036.jpg,motogp,13
11037,./train/11037.jpg,football,7
11038,./train/11038.jpg,football,7


In [11]:
class CustomDataset(Dataset):
    def __init__(self, file_csv, dir, transform=None, mode = 'train'):
        self.file_csv = file_csv
        self.dir = dir
        self.transform = transform
        self.mode = mode
            
    def __len__(self):
        return len(self.file_csv)
    
    #метод который позволяет нам индексировать датасет
    def __getitem__(self, idx):
        #считываем изображение
        image = cv2.imread(os.path.join(self.dir, self.file_csv['image'][idx]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        self.label = int(self.file_csv['num_labels'][idx])

        
        #применяем аугментации
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        if self.mode == 'train':
            return image, float(self.label)
        else:
            return image, self.file_csv['image'][idx]

In [77]:
#зададим немного гиперпараметров
batch_size = 32
num_workers = 2
img_size = 256

In [13]:
data_transforms = albumentations.Compose([
    albumentations.Resize(img_size, img_size),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.RandomBrightness(),
    albumentations.ShiftScaleRotate(rotate_limit=15, scale_limit=0.10),
    albumentations.HueSaturationValue(),
    albumentations.Normalize(),
    AT.ToTensor()
    ])

data_transforms_test = albumentations.Compose([
    albumentations.Resize(img_size, img_size),
    albumentations.HorizontalFlip(),
    albumentations.RandomRotate90(),
    albumentations.Normalize(),
    AT.ToTensor()
    ])

In [14]:
train_df, test_df = model_selection.train_test_split(
    train_data, test_size=0.15, random_state=42, stratify=train_data.num_labels.values
)
train_df, valid_df = model_selection.train_test_split(
    train_df, test_size=0.15, random_state=42, stratify=train_df.num_labels.values
)

train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [15]:
train_set = CustomDataset(train_df, train_dir, transform = data_transforms)
valid_set = CustomDataset(valid_df, train_dir, transform = data_transforms_test)
test_set = CustomDataset(test_df, train_dir, transform = data_transforms_test)

In [16]:
len(train_set), len(valid_set), len(test_set)

(7976, 1408, 1656)

In [81]:
trainloader = torch.utils.data.DataLoader(train_set, pin_memory=True, 
                                        batch_size=batch_size, shuffle=True, 
                                        num_workers=num_workers)
validloader = torch.utils.data.DataLoader(valid_set, pin_memory=True, 
                                        batch_size=batch_size, shuffle=True)

testloader = torch.utils.data.DataLoader(test_set, batch_size = batch_size,
                                        num_workers = num_workers)

In [ ]:
#Проверим работоспособность
samples, labels = next(iter(trainloader))
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(samples[:24])
np_grid_imgs = grid_imgs.numpy()
#  чтобы вывести изображение, нужно его немного преобразовать из тензора обратно в нампай
plt.imshow(np.transpose(np_grid_imgs, (1,2,0)))

In [82]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [20]:
# model = EfficientNet.from_pretrained('efficientnet-b3')
# #model

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth



Loaded pretrained weights for efficientnet-b3


In [21]:
model._fc = nn.Linear(in_features = 1536, out_features = 22)

In [ ]:
model = torchvision.models.resnet152(pretrained=True, progress=True)
#заморозим веса, чтобы использовать полностью предобученную сетку
for param in model.parameters():
    param.requires_grad = False

model

In [92]:
model.fc = nn.Linear(2048, 22)

In [85]:
def train_model(model_conv, train_loader, valid_loader, criterion, optimizer, scheduler, n_epochs):
    model_conv.to(device)
    valid_loss_min = np.Inf
    patience = 5
    # сколько эпох ждем до отключения
    p = 0
    # иначе останавливаем обучение
    stop = False

    # количество эпох
    for epoch in range(1, n_epochs+1):
        print(time.ctime(), 'Epoch:', epoch)

        train_loss = []

        for batch_i, (data, target) in enumerate(train_loader):

            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model_conv(data)
            loss = criterion(output, target.long())
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
    # запускаем валидацию
        model_conv.eval()
        correct = 0
        val_loss = []
        for batch_i, (data, target) in enumerate(valid_loader):
            data, target = data.to(device), target.to(device)
            output = model_conv(data)
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == target).sum().item()
            loss = criterion(output, target.long())
            val_loss.append(loss.item()) 
        
        acc = correct / len(valid_set)

        print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}.')
        print(f'Accuracy on valid set: {acc}')

        valid_loss = np.mean(val_loss)
        scheduler.step(valid_loss)
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model_conv.state_dict(), 'model.pt')
            valid_loss_min = valid_loss
            p = 0

        # проверяем как дела на валидации
        if valid_loss > valid_loss_min:
            p += 1
            print(f'{p} epochs of increasing val loss')
            if p > patience:
                print('Stopping training')
                stop = True
                break        

        if stop:
            break
    return model_conv, train_loss, val_loss

In [93]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.8, patience=2,)

In [94]:
model_resnet, train_loss, val_loss = train_model(model, trainloader, validloader, criterion, 
                              optimizer, scheduler, n_epochs=20,)

Wed Apr 28 14:49:34 2021 Epoch: 1
Epoch 1, train loss: 1.2515, valid loss: 0.9414.
Accuracy on valid set: 0.7215909090909091
Validation loss decreased (inf --> 0.941429).  Saving model ...
Wed Apr 28 14:53:44 2021 Epoch: 2
Epoch 2, train loss: 0.5768, valid loss: 0.7243.
Accuracy on valid set: 0.7954545454545454
Validation loss decreased (0.941429 --> 0.724324).  Saving model ...
Wed Apr 28 14:57:50 2021 Epoch: 3
Epoch 3, train loss: 0.4820, valid loss: 0.7983.
Accuracy on valid set: 0.7642045454545454
1 epochs of increasing val loss
Wed Apr 28 15:01:56 2021 Epoch: 4
Epoch 4, train loss: 0.4413, valid loss: 0.6970.
Accuracy on valid set: 0.7954545454545454
Validation loss decreased (0.724324 --> 0.697042).  Saving model ...
Wed Apr 28 15:06:04 2021 Epoch: 5
Epoch 5, train loss: 0.4201, valid loss: 0.6926.
Accuracy on valid set: 0.7947443181818182
Validation loss decreased (0.697042 --> 0.692585).  Saving model ...
Wed Apr 28 15:10:11 2021 Epoch: 6
Epoch 6, train loss: 0.3712, valid los

In [88]:
#загрузим лучшую модель и проведем на ней инференс (прогон тестовых данных)
model.state_dict(torch.load('/content/model.pt'))
print(1)

1


In [89]:
model.eval()
correct = 0

with torch.no_grad():
  for data, target in testloader:
    data = data.to(device=device)
    target = target.to(device=device)
    outputs = model(data)
    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted == target).sum().item()


acc = correct / len(test_set)
print(f'Accuracy on test set: {acc}')

Accuracy on test set: 0.8103864734299517


In [ ]:
#Проверим работоспособность
samples, labels = next(iter(testloader))
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(samples[:24])
np_grid_imgs = grid_imgs.numpy()
#  чтобы вывести изображение, нужно его немного преобразовать из тензора обратно в нампай
print(labels)
plt.imshow(np.transpose(np_grid_imgs, (1,2,0)))

In [ ]:
from pathlib import Path

model.to(device)
model.eval()
df_preds = pd.DataFrame()

file_dir = '/content/ttest'

name_files = os.listdir(file_dir)
for i in range(0, len(name_files)):
  name_files[i] = int(name_files[i].replace(".jpg", ""))
name_files.sort()
for i in range(0, len(name_files)):
  name_files[i] = str(name_files[i]) + ".jpg"

for i in range(0, len(name_files)):
    img = cv2.imread(file_dir + '/' + str(name_files[i]))[:, ::-1]
    img = data_transforms_test(image=img)['image'].cuda()
    pred = model(img[None])
    
    df_preds = df_preds.append(
        {'image': str(name_files[i]), 'labels': encoder.inverse_transform(torch.argmax(pred.cpu(), dim=1))[0]},
        ignore_index=True)

df_preds

In [ ]:
df_preds.to_csv('/content/submission.csv', index = False)

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/taskonML/ttest.zip', 'r') as zip_ref:
  zip_ref.extractall('/content')

with zipfile.ZipFile('/content/drive/MyDrive/taskonML/model.zip', 'r') as zip_ref:
  zip_ref.extractall('/content')

In [ ]:
weight = '/content/model.pt'
path_to_dataset = '/content/ttest/'

In [ ]:
def infer(weight, path_to_dataset):
  from pathlib import Path
  from sklearn.preprocessing import LabelEncoder

  img_size = 256

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = torchvision.models.resnet152(pretrained=True, progress=True)
  model.fc = nn.Linear(2048, 22)
  model.load_state_dict(torch.load(weight))

  name_files = os.listdir(path_to_dataset)
  for i in range(0, len(name_files)):
   name_files[i] = int(name_files[i].replace('.jpg', ''))
  name_files.sort()
  for i in range(0, len(name_files)):
   name_files[i] = str(name_files[i]) + '.jpg'

  labels_class = ['badminton',
                  'baseball',
                  'basketball',
                  'boxing',
                  'chess',
                  'cricket',
                  'fencing',
                  'football',
                  'formula1',
                  'gymnastics',
                  'hockey',
                  'ice_hockey',
                  'kabaddi',
                  'motogp',
                  'shooting',
                  'swimming',
                  'table_tennis',
                  'tennis',
                  'volleyball',
                  'weight_lifting',
                  'wrestling',
                  'wwe']

  data_transforms = albumentations.Compose([
    albumentations.Resize(img_size, img_size),
    albumentations.Normalize(),
    AT.ToTensor()
    ]) 
  
  model.to(device)
  model.eval()
  df_preds = pd.DataFrame()
  for i in range(0, len(name_files)):
    img = cv2.imread(path_to_dataset + '/' + str(name_files[i]))[:, ::-1]
    img = data_transforms(image=img)['image'].cuda()
    pred = model(img[None])
    
    df_preds = df_preds.append(
        {'image': str(name_files[i]), 'labels': labels_class[torch.argmax(pred.cpu(), dim=1)]},
        ignore_index=True)
  
  df_preds.to_csv('/content/submission.csv', index = False)

In [ ]:
infer(weight, path_to_dataset)